In [50]:
import psycopg2
import redis
import logging

In [51]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger('db_example')

In [41]:
lang_list = ['Python', 'JavaScript', 'R', 'Go']

In [56]:
try:
    conn = psycopg2.connect("dbname='halcyon' user='postgres' host='localhost' password='hi'")
except:
    print ("I am unable to connect to the database")
    
try:    
    r = redis.StrictRedis(host='localhost', port=6379, db=0)
except:
    print ("I am unable to connect to redis")

In [57]:
cur = conn.cursor()

In [34]:
lang = "Go"

In [35]:
cur.execute(  
"""select repo_name, language, num_stars, stars, stars::real / ((num_stars::real+400)^(1.6) ) AS normalized_stars
From 
(
select repo_id, repo_name, language, num_stars, count(*) AS stars from halcyon."Test_Hourly_Watches" 
Inner Join halcyon."Test_Repos" On repo_id = id
where num_stars > 0 
group by repo_id, repo_name, language, num_stars order by num_stars desc
) AS x 
where language Like '%s'
order by normalized_stars desc""" %lang )

In [36]:
rows = cur.fetchall()

In [37]:
rows[:10]

[('intelsdi-x/snap', 'Go', 486, 227, 0.0043664842920849),
 ('nishanths/license', 'Go', 163, 109, 0.00433124814722894),
 ('mvdan/interfacer', 'Go', 94, 83, 0.00406550944259128),
 ('RobotsAndPencils/buford', 'Go', 131, 92, 0.00401454743320943),
 ('Netflix/rend', 'Go', 147, 93, 0.0038699310419301),
 ('golang-china/gopl-zh', 'Go', 403, 158, 0.00355723219689982),
 ('deferpanic/gorump', 'Go', 133, 75, 0.00325310236168567),
 ('mission-liao/dingo', 'Go', 58, 57, 0.00315130247730193),
 ('apex/apex', 'Go', 91, 63, 0.00311609121567257),
 ('ory-am/hydra', 'Go', 499, 161, 0.0030255920423495)]

### Insert into Redis

#### Delete the set before updating keys

In [39]:
pipe = r.pipeline() # pipelines buffer multiple commands to the server in a single request
pipe.delete('%s:Weekly'%lang) # delete the set before updating the keys
for row in rows[:10]:
    print ("   ", row)
    pipe.zadd('%s:Weekly'%lang, row[4], row[0])

pipe.execute()

    ('SpectoLabs/hoverfly', 'Go', 153, 71, 0.00290334161188488)
    ('xenolf/lego', 'Go', 823, 237, 0.00272186145042855)
    ('kulshekhar/fungen', 'Go', 56, 45, 0.00250535207491604)
    ('raphael/goa', 'Go', 342, 95, 0.00242704236309749)
    ('kobolog/gorb', 'Go', 280, 82, 0.00240879361907017)
    ('goji/goji', 'Go', 47, 40, 0.00229915343103032)
    ('kavehmz/queue', 'Go', 59, 41, 0.00225883006657422)
    ('graphql-go/graphql', 'Go', 231, 66, 0.00218522902708545)
    ('klauspost/asmfmt', 'Go', 48, 38, 0.00217640028557615)
    ('vcrypt/vcrypt', 'Go', 48, 36, 0.00206185290212478)


[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [43]:
for lang in lang_list:
    cur = conn.cursor()
    cur.execute(  
    """select repo_name, language, num_stars, stars, stars::real / ((num_stars::real+400)^(1.6) ) AS normalized_stars
    From 
    (
    select repo_id, repo_name, language, num_stars, count(*) AS stars from halcyon."Test_Hourly_Watches" 
    Inner Join halcyon."Test_Repos" On repo_id = id
    where num_stars > 0 
    group by repo_id, repo_name, language, num_stars order by num_stars desc
    ) AS x 
    where language Like '%s'
    order by normalized_stars desc""" %lang )
    rows_400_6 = cur.fetchall()
    
    pipe = r.pipeline() # pipelines buffer multiple commands to the server in a single request
    pipe.delete('%s:curr_week'%lang) # delete the set before updating the keys
    for row in rows_400_6[:10]:
        print ("   ", row)
        pipe.zadd('%s:curr_week'%lang, row[4], row[0])

    pipe.execute()

    ('conan-io/conan', 'Python', 198, 127, 0.0045822821457416)
    ('OrkoHunter/ping-me', 'Python', 144, 99, 0.00415601355236504)
    ('dlapiduz/letsencrypt-s3front', 'Python', 74, 69, 0.00361080257964819)
    ('aantonop/wifiportal21', 'Python', 45, 57, 0.00329988525069339)
    ('v1k45/django-notify-x', 'Python', 71, 62, 0.00327761726103564)
    ('jmathai/elodie', 'Python', 162, 81, 0.00322780220525013)
    ('zero-db/zerodb-server', 'Python', 109, 65, 0.00303504339439264)
    ('RackHD/RackHD', 'Python', 79, 58, 0.00298463461904136)
    ('asrivat1/DeepLearningVideoGames', 'Python', 158, 73, 0.00294244356388596)
    ('arrayfire/arrayfire-python', 'Python', 116, 63, 0.00287806784141272)
    ('vuejs/vuex', 'JavaScript', 381, 216, 0.00508407673641913)
    ('trailsjs/trails', 'JavaScript', 191, 126, 0.00463266157935211)
    ('yelouafi/redux-saga', 'JavaScript', 292, 161, 0.00459892285895504)
    ('Microsoft/node', 'JavaScript', 75, 88, 0.00458957949692293)
    ('justinfagnani/mixwith.js', 'J

### Populate Caches for each Language and Variable Combo 

In [48]:
for lang in lang_list: 
    for base_stars, exponent in [[400, 6], [100, 6], [400, 8]]:
        print('currently processing weekly for: ', base_stars, exponent)
        pipe = r.pipeline()
        # current week's
        pipe.delete('%s:curr_week_%s_%s'%(lang, base_stars, exponent))
        cur = conn.cursor()
        cur.execute(  
            """select repo_name, language, num_stars, stars, stars::real / ((num_stars::real+%s)^(1.%s) ) AS normalized_stars
            From 
            (
                select repo_id, repo_name, language, num_stars, count(*) AS stars from halcyon."Test_Hourly_Watches" 
                Inner Join halcyon."Test_Repos" On repo_id = id
                where num_stars > 0 
                AND date > '2015-12-24'
                AND date <=  '2015-12-31'
                group by repo_id, repo_name, language, num_stars order by num_stars desc
            ) AS x 
            where language Like '%s'
            order by normalized_stars desc""" %(base_stars, exponent, lang) )
        rows_curr = cur.fetchall()
        for row in rows_curr[:10]:
            pipe.zadd('%s:curr_week_%s_%s'%(lang, base_stars, exponent), row[4], row[0])
        
        # prev week's
        pipe.delete('%s:prev_week_%s_%s'%(lang, base_stars, exponent))
        cur = conn.cursor()
        cur.execute(  
            """select repo_name, language, num_stars, stars, stars::real / ((num_stars::real+%s)^(1.%s) ) AS normalized_stars
            From 
            (
                select repo_id, repo_name, language, num_stars, count(*) AS stars from halcyon."Test_Hourly_Watches" 
                Inner Join halcyon."Test_Repos" On repo_id = id
                where num_stars > 0 
                AND date > '2015-12-17'
                AND date <=  '2015-12-24'
                group by repo_id, repo_name, language, num_stars order by num_stars desc
            ) AS x 
            where language Like '%s'
            order by normalized_stars desc""" %(base_stars, exponent, lang) )
        rows_prev = cur.fetchall()
        for row in rows_prev[:10]:
            pipe.zadd('%s:prev_week_%s_%s'%(lang, base_stars, exponent), row[4], row[0])

        pipe.execute()
        

400 6
100 6
400 8
400 6
100 6
400 8
400 6
100 6
400 8
400 6
100 6
400 8


In [54]:
cur = conn.cursor()

In [58]:
for lang in lang_list: 
    for base_stars, exponent in [[400, 6], [100, 6], [400, 8]]:
        print('currently processing ', lang, ' weekly for: ', base_stars, exponent)
        pipe = r.pipeline()
        # current week's
        pipe.delete('%s:curr_week_%s_%s'%(lang, base_stars, exponent))
        cur = conn.cursor()
        cur.execute(  
            """select repo_name, language, num_stars, stars, stars::real / ((num_stars::real+%s)^(1.%s) ) AS normalized_stars
            From 
            (
                select repo_id, repo_name, language, num_stars, count(*) AS stars from halcyon."Test_Hourly_Watches" 
                Inner Join halcyon."Test_Repos" On repo_id = id
                where num_stars > 0 
                AND date > current_date - 47
                AND date <=  current_date - 40
                group by repo_id, repo_name, language, num_stars order by num_stars desc
            ) AS x 
            where language Like '%s'
            order by normalized_stars desc""" %(base_stars, exponent, lang) )
        rows_curr = cur.fetchall()
        for row in rows_curr[:10]:
            pipe.zadd('%s:curr_week_%s_%s'%(lang, base_stars, exponent), row[4], row[0])
        
        # prev week's
        pipe.delete('%s:prev_week_%s_%s'%(lang, base_stars, exponent))
        cur = conn.cursor()
        cur.execute(  
            """select repo_name, language, num_stars, stars, stars::real / ((num_stars::real+%s)^(1.%s) ) AS normalized_stars
            From 
            (
                select repo_id, repo_name, language, num_stars, count(*) AS stars from halcyon."Test_Hourly_Watches" 
                Inner Join halcyon."Test_Repos" On repo_id = id
                where num_stars > 0 
                AND date > current_date - 54
                AND date <=  current_date - 47
                group by repo_id, repo_name, language, num_stars order by num_stars desc
            ) AS x 
            where language Like '%s'
            order by normalized_stars desc""" %(base_stars, exponent, lang) )
        rows_prev = cur.fetchall()
        for row in rows_prev[:10]:
            pipe.zadd('%s:prev_week_%s_%s'%(lang, base_stars, exponent), row[4], row[0])

        pipe.execute()
        

currently processing weekly for:  400 6
currently processing weekly for:  100 6
currently processing weekly for:  400 8
currently processing weekly for:  400 6
currently processing weekly for:  100 6
currently processing weekly for:  400 8
currently processing weekly for:  400 6
currently processing weekly for:  100 6
currently processing weekly for:  400 8
currently processing weekly for:  400 6
currently processing weekly for:  100 6
currently processing weekly for:  400 8
